# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

A. Tzouvanou  ->  A. Tzouvanou  |  ['A. Tzouvanou']
B. Bitsch  ->  B. Bitsch  |  ['B. Bitsch']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 60 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2307.11172


extracting tarball to tmp_2307.11172...

 done.


/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.17/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3508: LatexWarning: Found documentclass in tmp_2307.11172/aanda.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


A. Tzouvanou  ->  A. Tzouvanou  |  ['A. Tzouvanou']
B. Bitsch  ->  B. Bitsch  |  ['B. Bitsch']


Found 0 bibliographic references in tmp_2307.11172/ms.bbl.
Found 65 bibliographic references in tmp_2307.11172/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2307.11270


extracting tarball to tmp_2307.11270... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.11172-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.11172) | **Do all gaps in protoplanetary discs host planets?**  |
|| <mark>A. Tzouvanou</mark>, <mark>B. Bitsch</mark>, G. Pichierri |
|*Appeared on*| *2023-07-24*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Following the assumption that the disc substructures observed in protoplanetary discs originate from the interaction between the disc and the forming planets embedded therein, we aim to test if these putative planets could represent the progenitors of the currently observed giant exoplanets. We performed N-body simulations assuming initially three, four, five or seven planets. Our model includes pebble and gas accretion, migration, damping of eccentricities and inclinations, disc-planet interaction and disc evolution. We locate the planets in the positions where the gaps in protoplanetary discs have been observed and we evolve the systems for 100Myr including a few Myr of gas disc evolution, while also testing three values of $\alpha$ viscosity. For planetary systems with initially three and four planets we find that most of the growing planets lie beyond the RV detection limit of 5AU and only a small fraction of them migrate into the inner region. We also find that these systems have too low final eccentricities to be in agreement with the observed giant planet population. Systems initially consisting of five or seven planets become unstable after $\approx$40Kyr of integration time. This clearly shows that not every gap can host a planet. The general outcome of our simulations - too low eccentricities - is independent of the disc's viscosity and surface density. Further observations could either confirm the existence of an undetected population of wide-orbit giants or exclude the presence of such undetected population to constrain how many planets hide in gaps even further. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2307.11270-b31b1b.svg)](https://arxiv.org/abs/arXiv:2307.11270) | **The Role of $r$-Modes in Pulsar Spindown, Pulsar Timing and  Gravitational Waves**  |
|| V. Upadhyaya, X. Li, <mark>X. Zhang</mark>, S. R. Valluri |
|*Appeared on*| *2023-07-24*|
|*Comments*| **|
|**Abstract**| Pulsars are fast spinning neutron stars that lose their rotational energy via various processes such as gravitational and magnetic radiation, particle acceleration and mass loss processes. This dissipation can be quantified by a spin-down equation that measures the rate of change of the frequency as a function of the rotational frequency itself. We explore the pulsar spin-down and consider the spin-down equation upto the seventh order in frequency. This seventh order term accounts for energy loss due to the gravitational radiation caused by a current type quadrupole in the pulsar due to $r$-modes. We derive the rotational frequency due to the $r$-modes and find a solution in terms of the Lambert function. We also present an analytic exact solution for the period from the spindown equation and numerically verify this for the Crab pulsar. This analysis will be relevant for the detection of continuous gravitational waves by 3G ground based and space based gravitational wave detectors. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2307.11172.md
    + _build/html/tmp_2307.11172/./mass-semiaxis_3pl.png
    + _build/html/tmp_2307.11172/./cumulative_ecc_566_3.png
    + _build/html/tmp_2307.11172/./mass-semiaxis_4pl.png
    + _build/html/tmp_2307.11172/./cumulative_ecc_566_4.png
    + _build/html/tmp_2307.11172/./semiaxis-time_7pl.png
    + _build/html/tmp_2307.11172/./ec-time_7pl.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\GP}[1]{\textcolor{green}{#1}}$</div>



<div id="title">

# Do all gaps in protoplanetary discs host planets?

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2307.11172-b31b1b.svg)](https://arxiv.org/abs/2307.11172)<mark>Appeared on: 2023-07-24</mark> -  _Accepted for publication in A&A_

</div>
<div id="authors">

<mark>A. Tzouvanou</mark>, <mark>B. Bitsch</mark>, G. Pichierri

</div>
<div id="abstract">

**Abstract:** Following the assumption that the disc substructures observed in protoplanetary discs originate from the interaction between the disc and the forming planets embedded therein, we aim to test if these putative planets could represent the progenitors of the currently observed giant exoplanets.  We performed N-body simulations assuming initially three, four, five or seven planets. Our model includes pebble and gas accretion, migration, damping of eccentricities and inclinations, disc-planet interaction and disc evolution. We locate the planets in the positions where the gaps in protoplanetary discs have been observed and we evolve the systems for 100Myr including a few Myr of gas disc evolution, while also testing three values of $\alpha$ viscosity.  For planetary systems with initially three and four planets we find that most of the growing planets lie beyond the RV detection limit of 5AU and only a small fraction of them migrate into the inner region. We also find that these systems have too low final eccentricities to be in agreement with the observed giant planet population.  Systems initially consisting of five or seven planets become unstable after $\approx$ 40Kyr of integration time. This clearly shows that not every gap can host a planet. The general outcome of our simulations - too low eccentricities - is independent of the disc's viscosity and surface density. Further observations could either confirm the existence of an undetected population of wide-orbit giants or exclude the presence of such undetected population to constrain how many planets hide in gaps even further.

</div>

<div id="div_fig1">

<img src="tmp_2307.11172/./mass-semiaxis_3pl.png" alt="Fig3.1" width="50%"/><img src="tmp_2307.11172/./cumulative_ecc_566_3.png" alt="Fig3.2" width="50%"/>

**Figure 3. -** The left figure shows the comparison between the observed giant exoplanet population (gray dots) and all the simulations for the three- planet systems. The initial values of these planets are denoted as orange for the case where the planets  located in a wide configuration while brown dots denote planets located in a compact configuration, where the small numbers indicate the position of the first, second, and third planet. Purple, green and red data points symbolize the simulations for the different $\mathrm{\alpha}$, $10^{-3}$, $3.16 \cdot 10^{-4}$ and $10^{-4}$ respectively, whereas the coloured numbers shows the mean value and the standard deviation of the eccentricity for each case for all the simulated three- planet systems. The horizontal lines refers to the perihelion and aphelion of the planet $\mathrm{a(1 \pm e)}$. The black dashed vertical line represents the current Radial Velocity (RV) detection limit at 5 AU. Right figure shows the cumulative frequency of the eccentricity for planets up to 5 AU and mass between $ \mathrm{0.5 \leq mass [\mathrm{M_{Jup}}] \leq 13 }$. We also note that these plots include both cases for the initial mass (fixed and random value).  (*fig:3pl*)

</div>
<div id="div_fig2">

<img src="tmp_2307.11172/./mass-semiaxis_4pl.png" alt="Fig4.1" width="50%"/><img src="tmp_2307.11172/./cumulative_ecc_566_4.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** Similar to Fig. \ref{fig:3pl}, but for  systems with initially four planets.  (*fig:4pl*)

</div>
<div id="div_fig3">

<img src="tmp_2307.11172/./semiaxis-time_7pl.png" alt="Fig6.1" width="50%"/><img src="tmp_2307.11172/./ec-time_7pl.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Evolution of two different individual systems for the 5 (top) and 7 (bottom) planet system. The plots shows the evolution of the semi-major axis (left) and the eccentricity (right) over time. (*fig:individual_systems*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2307.11172"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

232  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
